In [1]:
from sklearn.datasets import fetch_california_housing
california = fetch_california_housing()
x = california.data
y = california.target

- 조건
    1. train set/ val set/ test set 분리 (자율비율)
    2. hidden lyaer 최소 3개 사용 (노드 수 자율)
    3. early stopping 적용 (patience 7)
    4. validation_data 옵션에 validation set 적용

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [8]:
# 데이터 분리
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=1,test_size=0.3)

In [9]:
# 모양 확인
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((14448, 8), (6192, 8), (14448,), (6192,))

In [10]:
# 회귀,분류 데이터 확인
y_train

array([2.387, 1.711, 0.554, ..., 1.048, 1.407, 1.26 ])

In [12]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_train.max(),x_train.min(),x_test.max(),x_test.min()

(1.0000000000000002, 0.0, 1.0, -0.005005005005005003)

In [28]:
# clear session
keras.backend.clear_session()
# layer set
il = keras.layers.Input(shape=(8,))
hl1 = keras.layers.Dense(64,activation='relu')(il)
hl2 = keras.layers.Dense(64,activation='relu')(hl1)
hl3 = keras.layers.Dense(64,activation='relu')(hl2)
hl4 = keras.layers.Dense(32,activation='relu')(hl3)
hl5 = keras.layers.Dense(32,activation='relu')(hl4)
ol = keras.layers.Dense(1)(hl5)
# model set
model = keras.models.Model(inputs=il,outputs=ol)
# model compile
model.compile(loss='mse',optimizer='Adam',metrics=['mae'])
# model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8)]               0         
                                                                 
 dense (Dense)               (None, 64)                576       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 32)                1056      
                                                                 
 dense_5 (Dense)             (None, 1)                 33    

In [26]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'val_loss',patience=7,min_delta=0,restore_best_weights=True,verbose=1)

In [29]:
model.fit(x_train,y_train,epochs=100,validation_split=0.3,callbacks=[es])

Epoch 1/100
317/317 [==============================] - 3s 5ms/step - loss: 0.9673 - mae: 0.7215 - val_loss: 0.5912 - val_mae: 0.5505
Epoch 2/100
317/317 [==============================] - 1s 4ms/step - loss: 0.5636 - mae: 0.5528 - val_loss: 0.5637 - val_mae: 0.5149
Epoch 3/100
317/317 [==============================] - 2s 6ms/step - loss: 0.5194 - mae: 0.5240 - val_loss: 0.5111 - val_mae: 0.4956
Epoch 4/100
317/317 [==============================] - 2s 5ms/step - loss: 0.4814 - mae: 0.4995 - val_loss: 0.4784 - val_mae: 0.4947
Epoch 5/100
317/317 [==============================] - 1s 4ms/step - loss: 0.4729 - mae: 0.4915 - val_loss: 0.5223 - val_mae: 0.4837
Epoch 6/100
317/317 [==============================] - 1s 4ms/step - loss: 0.4625 - mae: 0.4846 - val_loss: 0.4768 - val_mae: 0.4702
Epoch 7/100
317/317 [==============================] - 2s 6ms/step - loss: 0.4566 - mae: 0.4804 - val_loss: 0.4602 - val_mae: 0.4589
Epoch 8/100
317/317 [==============================] - 1s 4ms/step - 

In [30]:
y_pred = model.predict(x_test)

194/194 [==============================] - 0s 2ms/step


In [34]:
np.around(y_pred[:20].flatten(),3), y_test[:20]

(array([3.364, 0.637, 2.746, 1.374, 3.033, 3.807, 2.694, 1.336, 1.323,
        1.462, 2.849, 1.361, 0.852, 2.705, 0.916, 1.32 , 4.263, 1.346,
        1.366, 3.453], dtype=float32),
 array([3.55 , 0.707, 2.294, 1.125, 2.254, 2.63 , 2.268, 1.662, 1.18 ,
        1.563, 2.938, 0.975, 0.635, 3.149, 0.924, 1.172, 4.85 , 1.419,
        1.885, 2.286]))

In [32]:
# no activation
# y_pred
# (array([2.08, 0.94, 2.61, 1.62, 2.79, 3.35, 3.08, 1.56, 1.75, 2.02, 2.14,
#         0.84, 1.29, 2.71, 1.31, 1.78, 3.82, 2.04, 1.48, 2.7 ],
#        dtype=float32),
# y_test
#  array([3.55 , 0.707, 2.294, 1.125, 2.254, 2.63 , 2.268, 1.662, 1.18 ,
#         1.563, 2.938, 0.975, 0.635, 3.149, 0.924, 1.172, 4.85 , 1.419,
#         1.885, 2.286]))

In [33]:
# hidden layer activation relu
# y_pred
# (array([3.364, 0.637, 2.746, 1.374, 3.033, 3.807, 2.694, 1.336, 1.323,
#         1.462, 2.849, 1.361, 0.852, 2.705, 0.916, 1.32 , 4.263, 1.346,
#         1.366, 3.453], dtype=float32),
#  y_test
#  array([3.55 , 0.707, 2.294, 1.125, 2.254, 2.63 , 2.268, 1.662, 1.18 ,
#         1.563, 2.938, 0.975, 0.635, 3.149, 0.924, 1.172, 4.85 , 1.419,
#         1.885, 2.286]))

활성함수 넣으니까 확실히 잘맞추네